# Highway Corridors 

Where are the opportunities for better transit service on highways? Seattle has a lot of highways with transit that are more express service style, which complement the local transit service. 

What highways would benefit from more complementary bus express service? If Caltrans were to launch its express bus line on the highways, where would you put it?

Draw quadrant? See which highway corridors have a lot of parallel routes per mile, but few competitive routes per mile? % parallel routes vs % competitive routes? Both are normalized metrics.

In [ ]:
import altair as alt
import branca
import geopandas as gpd
import intake
import pandas as pd

from IPython.display import Markdown, HTML

import setup_corridors_stats
from create_parallel_corridors import IMG_PATH, DATA_PATH
from shared_utils import geography_utils, map_utils, styleguide
from shared_utils import calitp_color_palette as cp

alt.themes.register("calitp_theme", styleguide.calitp_theme)

catalog = intake.open_catalog("./*.yml")

In [ ]:
operator_stats, hwy_stats = setup_corridors_stats.aggregated_transit_hwy_stats()
gdf = setup_corridors_stats.process_hwy_stats(hwy_stats)

In [ ]:
def labeling(word):
    rename_dict = {
        "pct_parallel": "% parallel routes",
        "pct_competitive": "% competitive routes"
    }
    
    if word in rename_dict.keys():
        word = rename_dict[word]
    else:
        word = word.replace('_', ' ').title()
        
    return word

def make_scatterplot(df, x_col, y_col):
    
    if "pct" in x_col:
        x_p50 = 0.5
        y_p50 = 0.5
    else:
        x_p50 = 0.1
        y_p50 = 0.1
    
    chart = (alt.Chart(df)
             .mark_circle(size=30, opacity=0.6)
             .encode(
                 x=alt.X(f"{x_col}:Q", title=labeling(x_col)),
                 y=alt.Y(f"{y_col}:Q", title=labeling(y_col)),
                 color=alt.Color("District:N"),
                 tooltip=["Route", "County", "District",
                          "num_parallel", "pct_parallel", "parallel_per_mi",
                          "num_competitive", "pct_competitive", "competitive_per_mi",
                 ]
             ).interactive()
             .properties(title = f"{labeling(x_col)} vs {labeling(y_col)}")
    )
    
    horiz_line = (alt.Chart(df.assign(y_p50=y_p50))
                  .mark_rule(strokeDash=[2,3])
                  .encode(
                      y="y_p50:Q",
                      color=alt.value("black")
                  )
    )
    
    vertical_line = (alt.Chart(df.assign(x_p50=x_p50))
                     .mark_rule(strokeDash=[2,3])
                     .encode(
                         x="x_p50:Q",
                         color=alt.value("black")
                     )
    )
    
    combined = chart + horiz_line + vertical_line
    return combined

In [ ]:
make_scatterplot(gdf[gdf.pct_parallel > 0], "pct_parallel", "pct_competitive")

In [ ]:
make_scatterplot(gdf[gdf.pct_parallel > 0], "parallel_per_mi", "competitive_per_mi")